# **EKF Simulation**
In questo notebook implementeremo un EKF avanzato con tecniche di correzione del coning, sculling delayed state.

In [ ]:
import numpy as np
import math
import matplotlib

import utilities as tools

## Utilities

### LPF 1 ordine esponenziale

$$\alpha = exp(-\frac{2\pi f_c}{f_s})$$

$$y_k = \alpha y_{k-1} + (1 - \alpha)x_k$$

In [ ]:
def calculate_alpha(fc, fs):  #fc = cutoff frequency | fs = sample frequency
    return math.exp(-((2*math.pi*fc) / (fs)))

def lpfFirstOrderExp(x, y_prev, alpha):    #x = new measure
    y = alpha * y_prev + (1 - alpha) * x
    lpfFirstOrderExp.y_prev = y

    return y

### LPF 2 ordine butterworth
$$y_k = b_0 x_k + b_1 x_{k-1} + b_2 x_{k-2} - a_1 y_{y-1} - a_2 y_{k-2}$$

### Coning Compensation

$$\Delta\theta _{k} = \omega dt$$
$$\Delta\theta _{corr} = \Delta\theta _{k-1} + \Delta\theta _{k} + \frac{1}{12}(\Delta\theta _{k-1} \times \Delta\theta _{k} )$$

In [ ]:
def coning_compensation(omega, dtheta_prev, dt):
    dtheta = omega * dt

    return dtheta_prev + dtheta + 1/12*(np.cross(dtheta_prev, dtheta))

### Sculling Compensation


$$\Delta v_{k} = a dt$$
$$\Delta v_{corr} = \Delta v_{k-1} + \Delta v_{k} + \frac{1}{2}(\Delta\theta _{k-1} \times \Delta v_{k} ) + \frac{1}{12}(\Delta\theta _{k-1} \times \Delta v_{k-1} )$$

In [ ]:
def sculling_compensation(acc, dv_prev, dtheta_prev, dt):
    dv = acc * dt

    return dv_prev + dv + 1/2*np.cross(dtheta_prev, dv) + 1/12*np.cross(dtheta_prev, dv_prev)

## EKF

$$\delta X = [\delta\theta, \delta p, \delta v, \delta b_g, \delta b_a ]   $$
$$\delta X ∈ ℝ^{15}

In [ ]:
x_0 = np.array([1,0,0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0]).transpose()    # STATI NOMINALI ->  p[3] v[3] q[4] b_g[3] b_a[3]  forse-> earth magnetic field[3] body magnetic field [3]

delta_x_0 = np.array([0,0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0]).transpose()      #error states ->delta_pos[3], delta_vel[3],  delta_theta[3] (non si linearizzano i quaternioni)  delta_b_g[3] , delta_b_a[3]


g = 9.81





In [ ]:
class EKF:
    def __init__(self, dt):
        self.x_prev = np.array([0,0,0, 0,0,0, 1,0,0,0, 0,0,0, 0,0,0]).transpose() 
        
        self.delta_x_prev = np.array([0,0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0]).transpose() 

        self.P_prev = np.zeros((15,15))
        sigma_acc = 0.03      # m/s^2 / sqrt(Hz)
        sigma_gyro = 0.001    # rad/s / sqrt(Hz)
        sigma_ba = 0.0001     # m/s^2 / sqrt(Hz)
        sigma_bg = 1e-6       # rad/s / sqrt(Hz)

        Q = np.diag([
            sigma_acc**2, sigma_acc**2, sigma_acc**2,       # accelerometro
            sigma_gyro**2, sigma_gyro**2, sigma_gyro**2,    # giroscopio
            sigma_ba**2, sigma_ba**2, sigma_ba**2,          # bias accel
            sigma_bg**2, sigma_bg**2, sigma_bg**2           # bias gyro
        ])

        #variabili appoggio per sculling e coning
        self.delta_theta_corr_prev = 0
        self.delta_vel_corr_prev = 0


    def _jacobians_error_state(self, q, a, w, dt):
        """
        Restituisce F e G dell’error-state EKF2 di PX4.
        q : quaternion [qw, qx, qy, qz]
        a : accelero (bias corrected) in body frame
        w : gyro (bias corrected) in body frame
        dt: timestep
        """

        # --- rotation matrix ---
        R = tools.quat_to_rot(q)

        # Numero stati errore PX4
        n = 15

        # Matrici
        F = np.zeros((n, n))
        G = np.zeros((n, 12))   # rumori: accel, gyro, bias_a, bias_g

        # ---------------------------------------------------------
        # BLOCCO POSIZIONE
        # δp_dot = δv
        # ---------------------------------------------------------
        F[0:3, 3:6] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO VELOCITÀ
        # δv_dot = -R[a]_× δθ  - R δb_a  + R n_a
        # ---------------------------------------------------------
        F[3:6, 6:9] = -R @ tools.skew(a)
        F[3:6, 9:12] = -R

        # rumore accelerometro
        G[3:6, 0:3] = R

        # ---------------------------------------------------------
        # BLOCCO ORIENTAZIONE
        # δθ_dot = -[ω]_× δθ - δb_g + n_g
        # ---------------------------------------------------------
        F[6:9, 6:9] = -tools.skew(w)
        F[6:9, 12:15] = -np.eye(3)

        # rumore gyro
        G[6:9, 3:6] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO BIAS ACCELERO
        # δb_a_dot = n_ba
        # ---------------------------------------------------------
        # modello: random walk → F = 0, G = I * sigma
        G[9:12, 6:9] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO BIAS GIRO
        # δb_g_dot = n_bg
        # ---------------------------------------------------------
        G[12:15, 9:12] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO VENTO in PX4:
        # δw_dot = 0 (stato costante)
        # ---------------------------------------------------------
        # Nessuna dinamica → tutte le Jacobiane restano 0

        # ---------------------------------------------------------
        # Discretizzazione: Φ = I + F * dt
        # ---------------------------------------------------------
        Phi = np.eye(n) + F * dt

        # Noise discretization: Gd = G * dt
        Gd = G * dt

        return Phi, Gd, F, G
    
    def predict(self, gyro_meas, acc_meas, dt):
        

        #correzione bias
        omega = gyro_meas - self.x_prev[10:13]
        acc = acc_meas - self.x_prev[13:16] + np.array([0,0,g]).transpose()

        #Compensation
        delta_theta_corr = coning_compensation(omega,self.delta_theta_corr_prev, dt)
        delta_vel_corr = sculling_compensation(acc, self.delta_vel_corr_prev, self.delta_theta_corr_prev, dt)

        #Stima stati nominali
        x = np.empty((16, 1), dtype=float)

        x[6:10] =  tools.quat_update(self.x_prev[6:10], delta_theta_corr)
        x[3:6] = self.x_prev[3:6] + tools.rotmat_body_to_ned(x[6:10]) @ delta_vel_corr
        x[0:3] = self.x_prev[0:3] + x[3:6] * dt

        x[10:13] = self.x_prev[10:13] + np.random.normal(scale=1e-6, size=3)
        x[13:16] = self.x_prev[13:16] + np.random.normal(scale=1e-7, size=3)

        self.delta_theta_corr_prev = delta_theta_corr
        self.delta_vel_corr_prev = delta_vel_corr
        self.x_prev = x


        #Stima error state

        Phi, Gd, F, G = self._jacobians_error_state(x[6:10], acc, omega, dt)

        P = Phi @ self.P_prev @ Phi.T + Gd @ self.Q @ Gd.T

        self.P_prev = P

        return x, P
        